<h1>Exploring the taste of NYC neighborhoods</h1>

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [3]:
!pip install folium
!pip install wget

     |████████████████████████████████| 102kB 8.0MB/s ta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [4]:
import numpy as np # library to handle data in a vectorized manner

import wget
import os

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pprint import pprint # data pretty printer

import requests # library to handle requests
from bs4 import BeautifulSoup  # library to handle web scraping

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from collections import Counter # count occurrences 

from sklearn.cluster import KMeans # import k-means from clustering stage

In [5]:
if os.path.exists('newyork_data.json'):
    os.remove('newyork_data.json')

wget.download('https://cocl.us/new_york_dataset/newyork_data.json')

print('\nData downloaded!')


Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
type(newyork_data)

dict

In [7]:
neighborhood_data = newyork_data['features']

In [8]:
#Creating data frame
columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
nyc_df = pd.DataFrame(columns = columns)
nyc_df

,Borough,Neighborhood,Latitude,Longitude


In [9]:

for data in neighborhood_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_df = nyc_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
nyc_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
#Finding coordinates of NewYork city using geolocator
address = 'New York City, NY'
location = None

# define an instance of the geocoder -> ny_explorer
while location == None:
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    except:
        pass
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nyc_df['Latitude'], nyc_df['Longitude'], nyc_df['Borough'], nyc_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#1c1c1c',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
CLIENT_ID = 'OEVQYGFMA22NPP0GVV4TEGS4CDNU2UWJXZK1FV2YV4ZLKKLH' # your Foursquare ID
CLIENT_SECRET = 'YEZ4GJ2OBG1ODWZ1PQ53EAMUTZFXHKM45ORK5OGEMOVV5PQ2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OEVQYGFMA22NPP0GVV4TEGS4CDNU2UWJXZK1FV2YV4ZLKKLH
CLIENT_SECRET:YEZ4GJ2OBG1ODWZ1PQ53EAMUTZFXHKM45ORK5OGEMOVV5PQ2


In [14]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)
category_results = requests.get(url).json()

In [24]:
# category_results['response']['categories'][0]

In [16]:
for i in range(len(category_results['response']['categories'])):
    print('_____________________________')
    print(category_results['response']['categories'][i]['name'])
    for key, value in category_results['response']['categories'][i].items():        
        print(key, len((value)))

_____________________________
Arts & Entertainment
id 24
name 20
pluralName 20
shortName 20
icon 2
categories 36
_____________________________
College & University
id 24
name 20
pluralName 23
shortName 19
icon 2
categories 23
_____________________________
Event
id 24
name 5
pluralName 6
shortName 5
icon 2
categories 12
_____________________________
Food
id 24
name 4
pluralName 4
shortName 4
icon 2
categories 92
_____________________________
Nightlife Spot
id 24
name 14
pluralName 15
shortName 9
icon 2
categories 7
_____________________________
Outdoors & Recreation
id 24
name 21
pluralName 21
shortName 21
icon 2
categories 62
_____________________________
Professional & Other Places
id 24
name 27
pluralName 27
shortName 12
icon 2
categories 43
_____________________________
Residence
id 24
name 9
pluralName 10
shortName 9
icon 2
categories 5
_____________________________
Shop & Service
id 24
name 14
pluralName 16
shortName 5
icon 2
categories 145
_____________________________
Travel & T

In [17]:
category_list = category_results['response']['categories']
len(category_list)

10

# Now let us explore & analyze Food category 

In [19]:
nyc_food = []
for i in range(len(category_list)):
     if category_list[i]['name'] == 'Food':
            food = category_list[i]['categories']
            for i in range(len(food)):
                nyc_food.append(food[i]['name']) 

In [20]:
print(len(nyc_food))
nyc_food

92


['Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Asian Restaurant',
 'Australian Restaurant',
 'Austrian Restaurant',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bangladeshi Restaurant',
 'Belgian Restaurant',
 'Bistro',
 'Breakfast Spot',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Cafeteria',
 'Café',
 'Cajun / Creole Restaurant',
 'Caribbean Restaurant',
 'Caucasian Restaurant',
 'Coffee Shop',
 'Comfort Food Restaurant',
 'Creperie',
 'Czech Restaurant',
 'Deli / Bodega',
 'Dessert Shop',
 'Diner',
 'Donut Shop',
 'Dumpling Restaurant',
 'Dutch Restaurant',
 'Eastern European Restaurant',
 'English Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Fish & Chips Shop',
 'Fondue Restaurant',
 'Food Court',
 'Food Stand',
 'Food Truck',
 'French Restaurant',
 'Fried Chicken Joint',
 'Friterie',
 'Gastropub',
 'German Restaurant',
 'Gluten-free Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Hawaiian Restaurant',
 'Hot Dog Joint',
 'Hungar

<h4> Lets search for 'Food' within 500 meters for all neighborhoods of New York </h4>

In [21]:
radius = 500
LIMIT = 100


url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius,
            "4d4b7105d754a06374d81259", # "Food" category id
            LIMIT)
url

'https://api.foursquare.com/v2/venues/search?&client_id=OEVQYGFMA22NPP0GVV4TEGS4CDNU2UWJXZK1FV2YV4ZLKKLH&client_secret=YEZ4GJ2OBG1ODWZ1PQ53EAMUTZFXHKM45ORK5OGEMOVV5PQ2&v=20180605&ll=40.7127281,-74.0060152&radius=500&categoryId=4d4b7105d754a06374d81259&limit=100'

<h4> Let us explore the first neighborhood to understand the response format of GET request </h4>

In [26]:
#Fetch first neighborhoods name for testing
nyc_df.loc[0,'Neighborhood']

'Wakefield'

In [28]:
#Fetch Wakefield location details

neighborhood_lat = nyc_df.loc[0, 'Latitude']
neighborhood_lon = nyc_df.loc[0, 'Longitude']
neighborhood_name = nyc_df.loc[0, 'Neighborhood']
print(f'Title - {neighborhood_name}, Location - [{neighborhood_lat}, {neighborhood_lon}]')

Title - Wakefield, Location - [40.89470517661, -73.84720052054902]


In [40]:
#Now let's fetch the the nearby FOOD category venues within 500 meters from Foursquare API

LIMIT = 1
radius = 500
category_id = '4d4b7105d754a06374d81259'

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lon, 
    radius,
    category_id,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?&client_id=OEVQYGFMA22NPP0GVV4TEGS4CDNU2UWJXZK1FV2YV4ZLKKLH&client_secret=YEZ4GJ2OBG1ODWZ1PQ53EAMUTZFXHKM45ORK5OGEMOVV5PQ2&v=20180605&ll=40.89470517661,-73.84720052054902&radius=500&categoryId=4d4b7105d754a06374d81259&limit=1'

In [41]:
wakefield_results = requests.get(url).json()
wakefield_results

{'meta': {'code': 200, 'requestId': '5ec895e1b1cac0001b0c2482'},
 'response': {'venues': [{'id': '4c783cef3badb1f7e4244b54',
    'name': 'Carvel Ice Cream',
    'location': {'address': '1006 E 233rd St',
     'lat': 40.890486685759605,
     'lng': -73.84856772568665,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.890486685759605,
       'lng': -73.84856772568665},
      {'label': 'entrance', 'lat': 40.890438, 'lng': -73.848559}],
     'distance': 483,
     'postalCode': '10466',
     'cc': 'US',
     'city': 'Bronx',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1006 E 233rd St',
      'Bronx, NY 10466',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c9941735',
      'name': 'Ice Cream Shop',
      'pluralName': 'Ice Cream Shops',
      'shortName': 'Ice Cream',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-159

<h4>Now that we understood the response format of venue data. Let's fetch all venue details of all Neighborhoods</h4>

In [44]:
 venues_df_columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
venues_df = pd.DataFrame(columns = venues_df_columns)    

In [43]:
venues_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [45]:
nyc_df.shape

(306, 4)

In [52]:
#Create a function to loop the list of neighborhoods to fetch the nearby venues using the Foursquare API
def getNearbyVenues(name, latitude, longitude):
    """
        This method loops the nyc_df neighborhoods dataframe and fetches FOOD category venues within a range of 500 meters
    """
    
    radius = 500
    LIMIT = 100
    category_id = '4d4b7105d754a06374d81259' #Food category ID
    venue_list = []
    null_response_count = 0
    
    for neighborhood_name, neighborhood_lat, neighborhood_lon in zip(name, latitude, longitude):
        
        #create Foursquare
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_lat, 
        neighborhood_lon, 
        radius,
        category_id,
        LIMIT)
        
    #Now let's make a GET request to Foursquare API    
    try:
        results = requests.get(url).json()
        results = results['response']['venues']
        
        #Fetch venue details
        for venue in results:
            venue_list.append(neighborhood_name, neighborhood_lat, neighborhood_lon, 
                              venue['name'], venue['location']['lat'], venue['location']['lon'], venue['categories'][0]['name'])
    
    except:
        null_response_count += 1
        
    
    nearby_venues = pd.DataFrame([item for venue in venue_list for item in venue])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print(f'Total count of incomplete venues - {null_response_count}.')
    return(nearby_venues)
        

In [54]:
#lets import pickle and serialize the dataframe to a file for future use
import pickle

try:
    #This try block executes only when we have the file nyc_venues.pickle
    with open('nyc_venues.pkl', 'rb') as venue_list:
        nyc_venue_df = pickle.load(venue_list)
except:
    #This block executes when there's no file availble to read
    
    #Lets get the venue list data for every neighborhood
    nyc_venue_df = getNearbyVenues(name = nyc_df['Neighborhood'], latitude = nyc_df['Latitude'], longitude = nyc_df['Longitude'])
    
    #Once you receive the data, let's dump it a file using pickle module
    with open('nyc_venues.pkl', 'wb') as venue_list:
        pickle.dump(nyc_venue_df, venue_list)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements